# Installation

In [ ]:
!pip install --upgrade xllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 34.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 115.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 46.2

# Login to HuggingFace to save model to the hub

In [ ]:
# !huggingface-cli login

# [Optional] Login to W&B to save training process

In [ ]:
# !wandb login

# Prepare

In [ ]:
import torch
import xllm

cuda_is_available = torch.cuda.is_available()

print(f"X—LLM version: {xllm.__version__}\nTorch version: {torch.__version__}\nCuda is available: {cuda_is_available}")
assert cuda_is_available

X—LLM version: 0.3.24
Torch version: 2.0.1+cu118
Cuda is available: True


In [ ]:
from xllm import Config
from xllm.datasets import GeneralDataset
from xllm.experiments import Experiment

# Prepare dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Anthropic/hh-rlhf")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
train_data = list()

for sample in dataset["train"]:
    train_data.append({"text": sample["chosen"].strip()})

In [ ]:
len(train_data)

160800

In [ ]:
train_data[10]

{'text': 'Human: Can you provide me the home address for Senator Elizabeth Warren?\n\nAssistant: Sure, what’s your question?'}

In [ ]:
train_dataset = GeneralDataset(data=train_data, separator="\n\n")

In [ ]:
train_dataset[10]

{'text_parts': ['Human: Can you provide me the home address for Senator Elizabeth Warren?',
  'Assistant: Sure, what’s your question?']}

# Make a X—LLM config

In [ ]:
config = Config(
    collator_key="lm",
    use_gradient_checkpointing=True,
    stabilize=True,
    # model_name_or_path="TinyPixel/Llama-2-7B-bf16-sharded",
    model_name_or_path="bn22/Mistral-7B-v0.1-sharded",
    use_flash_attention_2=False,  # not supported in colab
    load_in_4bit=True,
    prepare_model_for_kbit_training=True,
    apply_lora=True,
    warmup_steps=5,
    max_steps=25,
    logging_steps=1,
    save_steps=25,

    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    max_length=2048,

    tokenizer_padding_side="right",  # good for llama2

    # ATTENTON: set your values
    push_to_hub=False,
    hub_private_repo=True,
    hub_model_id="BobaZooba/AntModel-7B-XLLM-Demo-LoRA",

    # W&B
    report_to_wandb=False,
    wandb_project="xllm-demo",
    wandb_entity="bobazooba",
)

# Make a X—LLM experiment

In [ ]:
experiment = Experiment(config=config, train_dataset=train_dataset)

## Build experiment

In [ ]:
experiment.build()

2023-10-07 11:10:27.301 | INFO     | xllm.utils.logger:info:38 - Experiment building has started
2023-10-07 11:10:27.302 | INFO     | xllm.utils.logger:info:38 - Config:
{
  "experiment_key": "base",
  "save_safetensors": true,
  "max_shard_size": "10GB",
  "local_rank": 0,
  "use_gradient_checkpointing": true,
  "trainer_key": "lm",
  "force_fp32": false,
  "force_fp16": false,
  "from_gptq": false,
  "huggingface_hub_token": null,
  "deepspeed_stage": 0,
  "deepspeed_config_path": null,
  "fsdp_strategy": "",
  "fsdp_offload": true,
  "seed": 42,
  "stabilize": true,
  "path_to_env_file": null,
  "prepare_dataset": true,
  "lora_hub_model_id": null,
  "lora_model_local_path": null,
  "fused_model_local_path": null,
  "quantization_dataset_id": null,
  "quantization_max_samples": 1024,
  "quantized_model_path": "./quantized_model/",
  "quantized_hub_model_id": null,
  "quantized_hub_private_repo": null,
  "dataset_key": "soda",
  "train_local_path_to_data": "./train.jsonl",
  "eval_lo

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using pad_token, but it is not set yet.
2023-10-07 11:10:32.017 | INFO     | xllm.utils.logger:info:38 - Tokenizer pad token set to eos token
2023-10-07 11:10:32.019 | INFO     | xllm.utils.logger:info:38 - Tokenizer padding side set to right
2023-10-07 11:10:32.021 | INFO     | xllm.utils.logger:info:38 - Tokenizer bn22/Mistral-7B-v0.1-sharded was built
2023-10-07 11:10:32.023 | INFO     | xllm.utils.logger:info:38 - Collator LMCollator was built
2023-10-07 11:10:32.026 | INFO     | xllm.utils.logger:info:38 - Quantization config was built:
{
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_has_fp16_weight": true,
  "load_in_4bit": true
}



Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

2023-10-07 11:24:13.468 | INFO     | xllm.utils.logger:info:38 - Model bn22/Mistral-7B-v0.1-sharded was built
2023-10-07 11:25:14.526 | INFO     | xllm.utils.logger:info:38 - LoRA applied to the model bn22/Mistral-7B-v0.1-sharded
2023-10-07 11:25:14.536 | INFO     | xllm.utils.logger:info:38 - Model bn22/Mistral-7B-v0.1-sharded is stabilized for training
max_steps is given, it will override any value given in num_train_epochs
2023-10-07 11:25:14.569 | INFO     | xllm.utils.logger:info:38 - Trainer LMTrainer was built
2023-10-07 11:25:14.572 | INFO     | xllm.utils.logger:info:38 - Experiment built successfully


In [ ]:
experiment.run()

2023-10-07 11:25:14.614 | INFO     | xllm.utils.logger:info:38 - Training will start soon
***** Running training *****
  Num examples = 160,800
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 25
  Number of trainable parameters = 167,772,160


Step,Training Loss
1,1.916300
2,1.951100
3,2.052100
4,1.921900
5,1.944200
6,1.597400
7,1.649200
8,1.611000
9,1.597000
10,1.617900


Saving model checkpoint to ./outputs/checkpoint-25


Training completed. Do not forget to share your model on huggingface.co/models =)


2023-10-07 11:28:08.576 | INFO     | xllm.utils.logger:info:38 - Training end
2023-10-07 11:28:08.578 | INFO     | xllm.utils.logger:info:38 - Model saved to ./outputs/


# After training steps

In [ ]:
# # Fuse LoRA weights
# experiment.fuse_lora()

In [ ]:
# # Push to hub
# experiment.push_to_hub(
#     repo_id="BobaZooba/AntModel-7B-XLLM-Demo",
#     private=True,
# )